<a href="https://colab.research.google.com/github/sahug/ds-bert/blob/main/BERT%20NLP%20-%20Text%20Summarization%20using%20BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**BERT NLP - Text Summarization using BERT**

**Summarization** creates a shorter version of a document or an article that captures all the important information. Along with translation, it is another example of a task that can be formulated as a sequence-to-sequence task. Summarization can be:

- **Extractive**: extract the most relevant information from a document.
- **Abstractive**: generate new text that captures the most relevant information.

**Load Dataset**

In [1]:
%pip install -qq datasets

     |████████████████████████████████| 346 kB 4.1 MB/s 
     |████████████████████████████████| 140 kB 25.0 MB/s 
     |████████████████████████████████| 1.1 MB 10.1 MB/s 
     |████████████████████████████████| 212 kB 43.7 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 86 kB 4.0 MB/s 
     |████████████████████████████████| 596 kB 52.7 MB/s 
     |████████████████████████████████| 127 kB 44.4 MB/s 
     |████████████████████████████████| 94 kB 1.5 MB/s 
     |████████████████████████████████| 144 kB 40.7 MB/s 
     |████████████████████████████████| 271 kB 49.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
from datasets import load_dataset
billsum = load_dataset("billsum", split="ca_test")

Using custom data configuration default


Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

Dataset billsum downloaded and prepared to /root/.cache/huggingface/datasets/billsum/default/3.0.0/d1e95173aed3acb71327864be74ead49b578522e4c7206048b2f2e5351b57959. Subsequent calls will reuse this data.


In [3]:
billsum

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 1237
})

**Look at Dataset**

In [4]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(
                lambda x: [typ.feature.names[i] for i in x]
            )
    display(HTML(df.to_html()))

In [7]:
show_random_elements(billsum)

**Train and Test Split**

In [8]:
billsum = billsum.train_test_split(test_size=0.2)

In [9]:
billsum["train"][0]

{'summary': 'Existing law generally prohibits strip searches and body cavity searches of prearraignment detainees arrested for infraction or misdemeanor offenses. Existing law allows a person who has been arrested and taken into custody to be subjected to patdown searches, metal detector searches, and thorough clothing searches in order to discover and retrieve concealed weapons and contraband substances prior to being placed in a booking cell.\nThis bill would also allow law enforcement personnel to subject a person who is arrested and taken into custody to a body scanner search for those weapons or substances. The bill would require an agency utilizing a body scanner to endeavor to avoid knowingly using a body scanner to scan a woman who is pregnant. The bill would require a person within sight of the visual display of a body scanner depicting the body during a scan to be of the same sex as the person being scanned, except for physicians or licensed medical personnel.',
 'text': 'The

**Preprocess**

The preprocessing function needs to:

- Prefix the input with a prompt so `T5` knows this is a summarization task. Some models capable of multiple NLP tasks require prompting for specific tasks.
- Use a context manager with the `as_target_tokenizer()` function to parallelize tokenization of inputs and labels.
- Truncate sequences to be no longer than the maximum length set by the `max_length` parameter.

In [10]:
%pip install -qq transformers

     |████████████████████████████████| 4.2 MB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 48.7 MB/s 


In [11]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:161: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [12]:
prefix = "summarize:"

def preprocess_function(examples):
  inputs = [prefix + doc for doc in examples["text"]]
  model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples["summary"], max_length=128, truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [13]:
pp = preprocess_function(billsum["train"][0])
print(pp["input_ids"][0])
print(pp["attention_mask"][0])
print(pp["labels"][0])

[21603, 10, 382, 1]
[1, 1, 1, 1]
17061


In [14]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Use `DataCollatorForSeq2Seq` to create a batch of examples. It will also dynamically pad your text and labels to the length of the longest element in its batch, so they are a uniform length. While it is possible to pad your text in the tokenizer function by setting `padding=True`, dynamic padding is more efficient.

In [15]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, return_tensors="tf")

**Finetune**

To **fine-tune** a model in TensorFlow, start by converting your datasets to the `tf.data.Dataset` format with `to_tf_dataset`. Specify inputs and labels in columns, whether to shuffle the dataset order, batch size, and the data collator.

In [16]:
tf_train_set = tokenized_billsum["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = tokenized_billsum["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

**Optimizer**

In [17]:
from transformers import create_optimizer, AdamWeightDecay
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

**Model**

In [18]:
from transformers import TFAutoModelForSeq2SeqLM
model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-small")

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


**Complile**

In [19]:
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour, please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


**Fit**

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=1)